In [1]:
import sys
import os

# 获取当前 Notebook 的目录（根目录）
notebook_dir = os.getcwd()

# 将 ALMT/models 目录添加到 sys.path
sys.path.append(os.path.join(notebook_dir, "ALMT", "models"))
sys.path.append(os.path.join(notebook_dir, "ALMT", "core"))
sys.path.append(os.path.join(notebook_dir, "ALMT"))

In [10]:
del sys.modules['almt_layer']
del sys.modules['dataset']

In [2]:
import torch
from torch import nn
from almt_layer import Transformer,CrossTransformerEncoder, HhyperLearningEncoder, CrossTransformer, Transformer
from bert import BertTextEncoder
from einops import repeat
from dataset import MMDataset
from core.scheduler import GradualWarmupScheduler
from metric import MetricsTop


import numpy as np
import torch.optim as optim
from torchvision import datasets, models, transforms

from torch.utils.data import DataLoader, Dataset
import torch.optim.lr_scheduler as lr_scheduler
from PIL import Image

import time
from tqdm import tqdm
from torch.distributions import Bernoulli
import csv



2025-04-24 19:01:33.714981: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-24 19:01:33.935352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-24 19:01:38.364812: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [74]:
import gc

def clear_memory():
    torch.cuda.empty_cache()  # 释放 GPU 缓存
    gc.collect()  # 释放 CPU 内存

clear_memory()

In [4]:
train_dataset = MMDataset(dataset='chsims',mode='train', image_num=5, generate_num = 4)
valid_dataset = MMDataset(dataset='chsims',mode='test', image_num=5, generate_num = 1)

100%|██████████| 457/457 [02:30<00:00,  3.04it/s]


In [75]:
valid_dataset = MMDataset(dataset='chsims',mode='valid', image_num=5, generate_num = 1)

100%|██████████| 456/456 [02:35<00:00,  2.93it/s]


In [9]:
torch.save(train_dataset, '/scratch/song.xinwe/chsims/train_dataset.pt')
torch.save(valid_dataset, '/scratch/song.xinwe/chsims/valid_dataset.pt')

In [4]:
train_dataset = torch.load('/scratch/song.xinwe/chsims/train_dataset.pt')
valid_dataset = torch.load('/scratch/song.xinwe/chsims/valid_dataset.pt')

/tmp/ipykernel_147860/3428264005.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load('/scratch/song.xinwe/chsims/train_dataset.pt')
/tmp/ipykernel

In [76]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
val_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=8)

In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [78]:
class CrossAttentionTA(nn.Module):
    def __init__(self, dim=128, heads=8, mlp_dim=128):
        super().__init__()
        self.cross_attn = CrossTransformer(source_num_frames=8, tgt_num_frames=8, dim=dim, depth=1, heads=heads, mlp_dim=mlp_dim)

    def forward(self, h_t, h_a):
        out = self.cross_attn(h_t, h_a)  # [B, 9, D]
        return out  # keep [CLS] and body for downstream split

class GateController(nn.Module):
    def __init__(self, dim=128):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, 1),
            #nn.Sigmoid()
        )

    def forward(self, state):
        # GateController.forward 中添加
        temperature = 1.
        p = torch.sigmoid(self.fc(state) / temperature)  
        #p = self.fc(state)
        dist = Bernoulli(probs=p)
        action = dist.sample()  # [B, 1]
        log_prob = dist.log_prob(action)  # [B, 1]
        return action, log_prob, p
        

In [79]:
class EMALMTBlock(nn.Module):
    def __init__(self, dim=128, heads=8, mlp_dim=128, dropout=0.):
        super(EMALMTBlock, self).__init__()

        # 融合模块（ALMT核心）
        self.fusion = HhyperLearningEncoder(dim=dim, depth=1, heads=heads, dim_head=16, dropout=dropout)

        # 视觉生成模块（E步）
        self.visual_predictor = Transformer(
            num_frames=16,
            save_hidden=False,
            token_len=None,  # 直接输出 token_len 长度的序列
            dim=dim,
            depth=1,
            heads=heads,
            mlp_dim=mlp_dim,
            dropout=dropout
        )

        # 视觉反馈映射模块（M步）
        self.feedback_proj = nn.Sequential(
            nn.Linear(dim, dim),
            nn.GELU(),
            nn.Linear(dim, dim)
        )
        
        self.cross_ta = CrossAttentionTA(dim=dim, heads=heads, mlp_dim=mlp_dim)
        self.gate_controller = GateController(dim=2*dim)  # one per layer


    def forward(self, h_v_list, h_l, h_a, h_hyper_v, layer_idx):
        """
        输入：
        - h_v_list: List of vision encoder layer outputs
        - h_l, h_a: 融合输入
        - h_hyper_v: 上一层融合结果
        - layer_idx: 当前层 index，用于选择要更新的视觉层
        输出：
        - h_hyper_v: 当前层融合输出
        - h_v_list: 修正后的视觉层列表
        """
        # 获取 TA 融合的 [CLS] token 表示


        h_ta_full = self.cross_ta(h_l, h_a)  # [B, 9, D] with [CLS] + 8 tokens
        gate_input = torch.cat([h_hyper_v[:,0],h_ta_full[:, 0]] ,dim=-1) 
        h_ta = h_ta_full[:, 1:]  # [B, 8, D] - remaining tokens for gated fusion
        gate, log_prob, prob = self.gate_controller(gate_input)  # [B, 1]
        gate = gate.unsqueeze(2)  # [B, 1, 1]

        # Apply gate to h_ta
        h_ta_gated = gate * h_ta

        h_hyper_v = self.fusion([h_v_list[layer_idx]], h_l, h_a, h_hyper_v)
        
        h_hyper_v = h_hyper_v + h_ta_gated

        # === E步：基于融合结果预测视觉表示
        h_v_feature = self.visual_predictor(h_hyper_v)  # (B, token_len, dim)
        
        if layer_idx+1<len(h_v_list):
            # === M步：将 h_v_feature 映射并反馈更新 h_v_list[layer_idx]
            h_v_feedback = self.feedback_proj(h_v_feature)  # (B, token_len, dim)

            h_v_list[layer_idx+1] = h_v_list[layer_idx+1] + h_v_feedback
            

        #return h_hyper_v, h_v_list, h_v_feature, log_prob, gate
        return h_hyper_v, h_v_list, h_v_feature, log_prob, gate, prob





class VisionFeatureAggregator(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=8),
            num_layers=1
        )

    def forward(self, x):  
        return self.transformer(x)
    
    
class ResNetWithDropout(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        resnet18 = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(resnet18.children())[:-1])  # 去掉最后一层 FC
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.features(x)
        x = x.squeeze(-1).squeeze(-1)
        x = self.dropout(x)
        return x


class Model(nn.Module):
    def __init__(self, AHL_depth=3, fusion_layer_depth=4, bert_pretrained='bert-base-chinese'):
        super(Model, self).__init__()

        self.h_hyper = nn.Parameter(torch.ones(1, 8, 128))

        # 基础编码器
        self.bertmodel = BertTextEncoder(use_finetune=True, transformers='bert', pretrained=bert_pretrained)
        self.img_extractor = ResNetWithDropout(dropout_rate=0.1)
        self.vf_aggregator = VisionFeatureAggregator(dim=512)

        # 投影层
        self.proj_l0 = nn.Sequential(nn.Linear(768, 128),nn.Dropout(0.1))
        self.proj_a0 = nn.Sequential(nn.Linear(33, 128),nn.Dropout(0.1))
        self.proj_v0 = nn.Sequential(nn.Linear(512, 128),nn.Dropout(0.1))

        # 序列处理
        self.proj_l = Transformer(num_frames=39, save_hidden=False, token_len=8, dim=128, depth=1, heads=8, mlp_dim=128)
        self.proj_a = Transformer(num_frames=400, save_hidden=False, token_len=8, dim=128, depth=1, heads=8, mlp_dim=128)
        self.proj_v = Transformer(num_frames=5, save_hidden=False, token_len=8, dim=128, depth=1, heads=8, mlp_dim=128)

        

        # 视觉主干编码器（输出多个中间层）
        self.vision_encoder = Transformer(num_frames=8, save_hidden=True, token_len=None, dim=128, depth=AHL_depth-1, heads=8, mlp_dim=128)

        # 新增：多层 EMALMTBlock 替代 h_hyper_layer_v
        self.em_almt_blocks = nn.ModuleList([
            EMALMTBlock(dim=128, heads=8, mlp_dim=128, dropout=0.)
            for _ in range(AHL_depth)
        ])

        # 跨模态融合 & 情感预测
        self.fusion_layer = CrossTransformer(source_num_frames=8, tgt_num_frames=8, dim=128, depth=fusion_layer_depth, heads=8, mlp_dim=128)
        self.cls_head = nn.Linear(128, 1)
        
        self.vision_feature_extractor = nn.Sequential(
            Transformer(num_frames=8, save_hidden=False, token_len=1, dim=128, depth=1, heads=8, mlp_dim=64, dropout=0.),   
            #nn.Dropout(0.3)
        )

        
    def forward(self, x_visual, x_audio, x_text):
        
        log_probs, gates, probs = [], [], []

        b = x_visual.size(0)
        h_hyper_v = repeat(self.h_hyper, '1 n d -> b n d', b=b)

        # 视觉特征提取
        x_visual = self.img_extractor(x_visual.view(-1, 3, 224, 224))
        x_visual = x_visual.view(b, 5, 512)
        x_visual = self.proj_v0(self.vf_aggregator(x_visual))



        x_audio = self.proj_a0(x_audio)
        x_text = self.bertmodel(x_text)
        x_text = self.proj_l0(x_text)

        h_v = self.proj_v(x_visual)[:, :8]
        h_a = self.proj_a(x_audio)[:, :8]
        h_l = self.proj_l(x_text)[:, :8]
        
        
        
        h_v_list = list(self.vision_encoder(h_v))  # 多层视觉表示
        

        for i, block in enumerate(self.em_almt_blocks):
            h_hyper_v, h_v_list, h_v_feature, log_prob, gate, prob = block(h_v_list, h_l, h_a, h_hyper_v, layer_idx=i)
            log_probs.append(log_prob)
            gates.append(gate)
            probs.append(prob)

        # 情感预测
        feat = self.fusion_layer(h_hyper_v, h_v_list[-1])[:, 0]
        output = self.cls_head(feat)

        return output, self.vision_feature_extractor(h_v_feature)[:,0], log_probs, gates, probs


In [80]:
model = Model().to(device)

emo_loss_fn = torch.nn.MSELoss()

def SupervisedContrastiveLoss(h_v_cls, labels, sigma=0.1):
    """
    :param h_v_cls: (B, D) - visual embeddings after projection (final representation)
    :param labels: (B,) - continuous sentiment labels
    :param sigma: float - Gaussian kernel width (smaller = stricter similarity)
    :return: scalar contrastive loss
    """
    batch_size = h_v_cls.shape[0]

    # Normalize embeddings
    h_v_cls = torch.nn.functional.normalize(h_v_cls, dim=1)  # (B, D)

    # Cosine similarity matrix: (B, B)
    similarity_matrix = torch.cosine_similarity(h_v_cls.unsqueeze(1), h_v_cls.unsqueeze(0), dim=-1)

    # Compute label distance matrix
    labels = labels.contiguous().view(-1, 1)  # (B, 1)
    label_diff = labels - labels.T  # (B, B)

    # Gaussian weight based on label similarity
    weight_matrix = torch.exp(- (label_diff ** 2) / (2 * sigma ** 2)).to(h_v_cls.device)  # (B, B)

    # Exponential of cosine similarity
    exp_sim = torch.exp(similarity_matrix)  # (B, B)

    # Avoid self-comparison by masking diagonal to 0
    identity_mask = torch.eye(batch_size, device=h_v_cls.device)
    weight_matrix = weight_matrix * (1 - identity_mask)
    exp_sim = exp_sim * (1 - identity_mask)

    # Compute weighted contrastive loss
    numerator = torch.sum(exp_sim * weight_matrix, dim=1)  # (B,)
    denominator = torch.sum(exp_sim, dim=1) + 1e-8          # (B,)

    loss = -torch.log(numerator / denominator + 1e-8)       # (B,)
    return loss.mean()


/scratch/song.xinwe/MultiModal/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/song.xinwe/MultiModal/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/scratch/song.xinwe/MultiModal/lib/python3.8/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.

In [40]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total:,}")
    print(f"Trainable parameters: {trainable:,}")

# 示例用法
count_parameters(model)


Total parameters: 122,167,556
Trainable parameters: 122,167,556


In [81]:
def save_validation_metrics(epoch, metrics, save_path="metrics_log_chsims.csv"):
    """
    保存每轮验证的指标到 CSV 文件。
    
    Args:
        epoch (int): 当前 epoch
        metrics (dict): 包含各类验证指标的字典
        save_path (str): 保存路径
    """
    file_exists = os.path.exists(save_path)
    header = ["epoch"] + list(metrics.keys())

    with open(save_path, "a", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)
        writer.writerow([epoch] + [metrics[k] for k in header[1:]])


In [82]:
best_val_mae = float("inf")

def save_best_model(model, mae, save_path='best_model_chsims.pt'):
    global best_val_mae
    if mae < best_val_mae:
        best_val_mae = mae
        torch.save(model.state_dict(), save_path)
        print(f"✅ Saved best model (MAE = {mae:.4f}) to {save_path}")


In [83]:
# =========================
# 设置训练参数
# =========================
epochs = 200
train_mae, val_mae = [], []

optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4, weight_decay=1e-4)
scheduler_steplr = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=0.9 * epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=0.05 * epochs, after_scheduler=scheduler_steplr)


# =========================
# AverageMeter 类用于记录损失
# =========================
class AverageMeter(object):
    def __init__(self):
        self.value = 0
        self.value_avg = 0
        self.value_sum = 0
        self.count = 0

    def reset(self):
        self.value = 0
        self.value_avg = 0
        self.value_sum = 0
        self.count = 0

    def update(self, value, count):
        self.value = value
        self.value_sum += value * count
        self.count += count
        self.value_avg = self.value_sum / self.count

# =========================
# Train 函数
# =========================
def train(model, train_loader, optimizer, epoch):
    train_pbar = tqdm(enumerate(train_loader), total=len(train_loader), dynamic_ncols=True)
    losses = AverageMeter()
    l1s = AverageMeter()
    l2s = AverageMeter()
    rls = AverageMeter()

    y_pred, y_true = [], []

    model.train()
    moving_baseline = 0.6
    alpha = 0.9
    
    lambda_recon = 0.3
    

        
    lambda_rl = 0.5

    for cur_iter, data in train_pbar:
        img = data['images'].to(device)
        audio = data['audio'].to(device)
        text = data['text'].to(device)
        label = data['labels'].to(device).view(-1, 1)
        batchsize = img.shape[0]

        optimizer.zero_grad()

        output, h_v_generated, log_probs, gates, probs = model(img, audio, text)

        l1 = emo_loss_fn(output, label)
        l2 = SupervisedContrastiveLoss(h_v_generated, label)

        # === REINFORCE ===
        mae_clamped = l1.detach().clamp(0, 2)
        reward = 1.0 - mae_clamped / 2.0
        advantage = reward - moving_baseline
        loss_rl = sum([-(lp.squeeze() * advantage).mean() for lp in log_probs])
        moving_baseline = alpha * moving_baseline + (1 - alpha) * reward.item()

       

        # === 总损失 ===
        loss = l1 + lambda_recon * l2 + lambda_rl * loss_rl
        loss.backward()
        optimizer.step()

        losses.update(loss.item(), batchsize)
        l1s.update(l1.item(), batchsize)
        l2s.update(l2.item(), batchsize)
        rls.update(loss_rl.item(), batchsize)

        y_pred.append(output.cpu())
        y_true.append(label.cpu())

        train_pbar.set_description(f'train [Epoch {epoch}]')
        train_pbar.set_postfix({
            'loss': '{:.5f}'.format(losses.value_avg),
            'emo_loss': '{:.4f}'.format(l1s.value_avg),
            'visual_loss': '{:.4f}'.format(l2s.value_avg),
            'RL_loss': '{:.4f}'.format(rls.value_avg),
            'λ_recon': '{:.2f}'.format(lambda_recon),
            'λ_RL': '{:.2f}'.format(lambda_rl),
            'lr': '{:.2e}'.format(optimizer.param_groups[0]['lr'])
        }, refresh=False)

    pred, true = torch.cat(y_pred), torch.cat(y_true)
    mae = torch.mean(torch.abs(pred - true)).item()
    tqdm.write(f"train MAE: {mae:.4f}")
    train_mae.append(mae)



# =========================
# Evaluate 函数
# =========================
def evaluate(model, eval_loader, optimizer, epoch):
    metric = MetricsTop()
    test_pbar = tqdm(enumerate(eval_loader), total=len(eval_loader))
    losses = AverageMeter()
    l1s = AverageMeter()
    l2s = AverageMeter()

    y_pred, y_true = [], []

    model.eval()
    with torch.no_grad():
        for cur_iter, data in test_pbar:
            img = data['images'].to(device)
            audio = data['audio'].to(device)
            text = data['text'].to(device)
            label = data['labels'].to(device).view(-1, 1)
            batchsize = img.shape[0]

            output, h_v_generated, log_probs, gates, probs = model(img, audio, text)           

            l1 = emo_loss_fn(output, label)
            l2 = SupervisedContrastiveLoss(h_v_generated, label)

            
            lambda_recon = 0.3
            
                
            loss = l1 + lambda_recon * l2
            
            y_pred.append(output.cpu())
            y_true.append(label.cpu())

            losses.update(loss.item(), batchsize)
            l1s.update(l1.item(), batchsize)
            l2s.update(l2.item(), batchsize)


            test_pbar.set_description('eval')
            test_pbar.set_postfix({
                'loss': '{:.5f}'.format(losses.value_avg),
                'emo_loss': '{:.4f}'.format(l1s.value_avg),
                'visual_loss': '{:.4f}'.format(l2s.value_avg),
                'lambda_recon': '{:.2f}'.format(lambda_recon),
                'lr': '{:.2e}'.format(optimizer.param_groups[0]['lr'])
            }, refresh=False)

        pred, true = torch.cat(y_pred), torch.cat(y_true)
        eval_func = metric.getMetics("SIMS")
        eval_results = eval_func(pred, true)
        
        print(f"Evaluation Results:")
        for key,value in eval_results.items():
            print(f"{key}: {value:.4f}")
        
        gate_stats = torch.cat(gates, dim=1)  # [B, num_layers]
        gate_ratio = gate_stats.float().mean(dim=0).cpu().numpy()

        print('Gate Activation Ratio:', gate_ratio)
        
        save_validation_metrics(epoch,eval_results)
        save_best_model(model, eval_results['MAE'])


In [ ]:
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    evaluate(model, val_loader, optimizer, epoch)  
    scheduler_warmup.step()

train [Epoch 1]: 100%|██████████| 86/86 [01:20<00:00,  1.07it/s, loss=1.35668, emo_loss=0.7340, visual_loss=2.0604, RL_loss=0.0091, λ_recon=0.30, λ_RL=0.50, lr=0.00e+00]


train MAE: 0.7392


eval: 100%|██████████| 8/8 [00:09<00:00,  1.23s/it, loss=1.36490, emo_loss=0.7323, visual_loss=2.1087, lambda_recon=0.30, lr=0.00e+00]


Evaluation Results:
Mult_acc_2: 0.3048
Mult_acc_3: 0.3048
Mult_acc_5: 0.1513
F1_score: 0.4672
MAE: 0.7397
Corr: -0.0019
Gate Activation Ratio: [[       0.25]
 [      0.625]
 [        0.5]]
✅ Saved best model (MAE = 0.7397) to best_model_chsims.pt


train [Epoch 2]: 100%|██████████| 86/86 [01:18<00:00,  1.10it/s, loss=1.07665, emo_loss=0.4281, visual_loss=2.0582, RL_loss=0.0621, λ_recon=0.30, λ_RL=0.50, lr=1.00e-05]


...

train MAE: 0.0139


eval: 100%|██████████| 8/8 [00:11<00:00,  1.41s/it, loss=1.08058, emo_loss=0.4608, visual_loss=2.0658, lambda_recon=0.30, lr=7.03e-05]

Evaluation Results:
Mult_acc_2: 0.7083
Mult_acc_3: 0.5768
Mult_acc_5: 0.3728
F1_score: 0.7005
MAE: 0.4950
Corr: 0.4715
Gate Activation Ratio: [[      0.625]
 [          0]
 [          1]]



train [Epoch 79]: 100%|██████████| 86/86 [01:24<00:00,  1.02it/s, loss=0.39385, emo_loss=0.0004, visual_loss=1.2644, RL_loss=0.0283, λ_recon=0.30, λ_RL=0.50, lr=6.95e-05]


train MAE: 0.0141
